In [58]:
import pandas as pd
from utilities import to_float

from matplotlib import pyplot as plt
import numpy as np

In [41]:
DATA_PATH = 'data/'

In [42]:
df_tweets = pd.read_csv(DATA_PATH+'tweets.csv')

In [43]:
df_users = pd.read_csv(DATA_PATH+'users.csv')

In [44]:
df_users.head()

,id,name,lang,bot,created_at,statuses_count
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54.0
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3.0
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50.0
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085.0


In [45]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              11508 non-null  int64  
 1   name            11507 non-null  object 
 2   lang            11508 non-null  object 
 3   bot             11508 non-null  int64  
 4   created_at      11508 non-null  object 
 5   statuses_count  11109 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 539.6+ KB


Check if any user record has a null value in it
Extract only record with a null name (id = 1535)

In [46]:
df_null = df_users.isnull()
idx_null = df_users.index[df_null["name"] == True].tolist()
df_users.iloc[idx_null]

,id,name,lang,bot,created_at,statuses_count
1535,2166124159,NaN,en,0,2018-11-02 06:39:14,6566.0


In [48]:
df_users.duplicated(['id']).value_counts()

False    11508
dtype: int64

Print info about tweets file

In [49]:
df_tweets.head()

,id,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text
0,509354017856950272,327746321,0,0,0,0,0,0,2019-09-11 14:53:55,"If man is a little lower than angels, then ang..."
1,583002592529121280,333722906,1,0,0,0,0,1,2020-04-01 20:27:04,"""@BestWSHHVids: how do you say these words wit..."
2,461498835362013185,2379755827,0,0,0,0,0,1,2019-05-02 13:34:31,@LOLatComedy awsome
3,528808127366692864,466226882,0,0,0,0,0,0,2019-11-04 07:17:37,Stephen Hawkins: i buchi neri non esistono se ...
4,575336690904006656,1355537995,114,0,0,1,0,1,2020-03-11 16:45:31,RT @tibbs_montris: So ready for Wednesday!


In [50]:
df_tweets.info(null_counts=True)

C:\Users\al\AppData\Local\Temp\ipykernel_15108\3419106329.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_tweets.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13664696 entries, 0 to 13664695
Data columns (total 10 columns):
 #   Column          Non-Null Count     Dtype 
---  ------          --------------     ----- 
 0   id              13664694 non-null  object
 1   user_id         13447413 non-null  object
 2   retweet_count   13227562 non-null  object
 3   reply_count     13016818 non-null  object
 4   favorite_count  13017154 non-null  object
 5   num_hashtags    12607172 non-null  object
 6   num_urls        13016073 non-null  object
 7   num_mentions    12810531 non-null  object
 8   created_at      13664696 non-null  object
 9   text            13126975 non-null  object
dtypes: object(10)
memory usage: 1.0+ GB


In [51]:
df_tweets.size

136646960

## Verifica esistenza record duplicati
First, we drop  the duplicates by attributes

In [21]:
df_tweets_dupl = df_tweets.duplicated()
df_tweets_dupl.value_counts()

False    11712597
True      1952099
dtype: int64

## Drop duplicates

In [22]:
df_tweets_clear = df_tweets.drop_duplicates(keep = 'first')
df_tweets_clear.shape[0]

11712597

In [54]:
df_tweets_clear.duplicated().value_counts()

False    11712597
dtype: int64

In [23]:
df_tweets_clear.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11712597 entries, 0 to 13664695
Data columns (total 10 columns):
 #   Column          Non-Null Count     Dtype 
---  ------          --------------     ----- 
 0   id              11712595 non-null  object
 1   user_id         11495314 non-null  object
 2   retweet_count   11275463 non-null  object
 3   reply_count     11064719 non-null  object
 4   favorite_count  11065055 non-null  object
 5   num_hashtags    10655073 non-null  object
 6   num_urls        11063974 non-null  object
 7   num_mentions    10858432 non-null  object
 8   created_at      11712597 non-null  object
 9   text            11182415 non-null  object
dtypes: object(10)
memory usage: 983.0+ MB


## Verifica di duplicati su un sottoinsieme di colonne

### ID
The are a considerable amount of tweets with same id, but we can't consider them duplicates since the id has no semantic.

In [24]:
df_tweets_dupl = df_tweets_clear.duplicated(subset=['id']) # Stesso id
df_tweets_dupl.value_counts()

False    11672136
True        40461
dtype: int64

In [25]:
df_tweets_dupl = df_tweets_clear.duplicated(subset=['id', 'user_id']) # Stesso id e stesso user_id
df_tweets_dupl.value_counts()

False    11712237
True          360
dtype: int64

In [26]:
df_tweets_dupl = df_tweets_clear.duplicated(subset=['id', 'user_id', 'text']) # Stessi id, user_id e testo
df_tweets_dupl.value_counts()

False    11712336
True          261
dtype: int64

Considering the triple  `['user_id', 'text','created_at']`, we can see that about 10% of the data consists in duplicated records...

In [55]:
df_tweets_dupl = df_tweets_clear.duplicated(subset=['user_id', 'text','created_at'])
df_tweets_dupl.value_counts()

False    10665159
True      1047438
dtype: int64

... and we are not able to detect them if we include the `id` column.

In [27]:
df_tweets_dupl = df_tweets_clear.duplicated(subset=['id', 'user_id', 'created_at', 'text']) # Stessi id, user_id, testo e data di creazione
df_tweets_dupl.value_counts()

False    11712597
dtype: int64

Of these 1047438 records,

In [56]:
df_tweets_clear[df_tweets_clear.duplicated(subset=['user_id', 'text','created_at'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1047438 entries, 7530 to 13664690
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   id              1047438 non-null  object
 1   user_id         1047351 non-null  object
 2   retweet_count   889431 non-null   object
 3   reply_count     889619 non-null   object
 4   favorite_count  808821 non-null   object
 5   num_hashtags    807627 non-null   object
 6   num_urls        889509 non-null   object
 7   num_mentions    889575 non-null   object
 8   created_at      1047438 non-null  object
 9   text            1041423 non-null  object
dtypes: object(10)
memory usage: 87.9+ MB


In [68]:
duplicates_bool = df_tweets_clear.duplicated(subset=['user_id', 'text','created_at'], keep=False)
df_tweets_clear[duplicates_bool]

,id,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text,retweet_count_conv,reply_count_conv,favorite_count_conv,num_hashtags_conv,num_urls_conv,num_mentions_conv
6,79540398988595200,67900288,0,0,0,0,0,0,2016-06-12 13:28:03,Andy Murray look like he is in form an ready t...,0.0,0.0,0.0,0.0,0.0,0.0
26,357651699093086209,38679217,1,0,0,0,0,2,2018-07-20 00:03:04,RT @rowiematti: It is the first day of Galileo...,1.0,0.0,0.0,0.0,0.0,2.0
40,584530682094886913,413675359,1,0,2,0,0,0,2020-04-06 01:39:09,Yesterday was a great day of geocaching,1.0,0.0,2.0,0.0,0.0,0.0
41,565184038814494720,1252098630,1,0,0,0,0,0,2020-02-12 16:22:30,"When understanding someone's sorrow, take time...",1.0,0.0,0.0,0.0,0.0,0.0
45,136036147826125509,531157328,0,0.0,0.0,0.0,NaN,0,2019-10-18 15:51:53,Cocaine is God's way of saying you're making t...,0.0,0.0,0.0,0.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13664664,573005278599241728,553317912,0,0,0,0,0,0,2020-03-05 06:21:18,"She want my eggs, bacon, grits , and sausage.",0.0,0.0,0.0,0.0,0.0,0.0
13664666,511318938266136577,512771375,0,0,0,0,1,0,2019-09-17 01:01:49,Read the History of the Chiffons http://t.co...,0.0,0.0,0.0,0.0,1.0,0.0
13664685,838370060,466207205,0,0.0,09vtlcms9k,NaN,0.0,0,2019-11-03 16:59:57,A volte mi piacerebbe sdoppiarmi per potermi a...,0.0,0.0,-1.0,-1.0,0.0,0.0
13664688,idjn,243282862,NaN,NaN,NaN,NaN,0.0,NaN,2019-09-07 02:31:17,Nobody remembers the guy who finished second b...,-1.0,-1.0,-1.0,-1.0,0.0,-1.0


We can observe that the other columns of these duplicates have a considerable amount of null values. In order to keep the correct values, we proceed by performing a merge of the copies, keeping the non null value.
We first convert the counts and nums to numeric type, setting to -1 the meaningless values.
Then, the merge will proceed by taking the max over the duplicates.

In [59]:
for attr in ['retweet_count','reply_count','favorite_count','num_hashtags','num_urls','num_mentions']:
    df_tweets_clear[attr+'_conv'] = df_tweets_clear[attr].apply(to_float)

In [61]:
df_tweets_clear

,id,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text,retweet_count_conv,reply_count_conv,favorite_count_conv,num_hashtags_conv,num_urls_conv,num_mentions_conv
0,509354017856950272,327746321,0,0,0,0,0,0,2019-09-11 14:53:55,"If man is a little lower than angels, then ang...",0.0,0.0,0.0,0.0,0.0,0.0
1,583002592529121280,333722906,1,0,0,0,0,1,2020-04-01 20:27:04,"""@BestWSHHVids: how do you say these words wit...",1.0,0.0,0.0,0.0,0.0,1.0
2,461498835362013185,2379755827,0,0,0,0,0,1,2019-05-02 13:34:31,@LOLatComedy awsome,0.0,0.0,0.0,0.0,0.0,1.0
3,528808127366692864,466226882,0,0,0,0,0,0,2019-11-04 07:17:37,Stephen Hawkins: i buchi neri non esistono se ...,0.0,0.0,0.0,0.0,0.0,0.0
4,575336690904006656,1355537995,114,0,0,1,0,1,2020-03-11 16:45:31,RT @tibbs_montris: So ready for Wednesday!,114.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13664690,329829994169786368,220933018,0,0,0,0,0,0,2018-05-04 05:29:33,ESTA MANANA AUN ESTA MUY FRIO ! MIREN ESTO ! ...,0.0,0.0,0.0,0.0,0.0,0.0
13664691,588535254207467520,587491046,0,0,0,0,0,1,2020-04-17 02:51:53,"@warriors Congrats, maybe I'll be able to get ...",0.0,0.0,0.0,0.0,0.0,1.0
13664693,718157017970961,91781300,0,NaN,l25suv5,NaN,0.0,NaN,2016-07-10 22:43:09,NaN,0.0,-1.0,-1.0,-1.0,0.0,-1.0
13664694,441301348676415488,127895572,0,0,1,1,0,0,2019-03-07 19:56:55,Shooting crew of porn movies. #TheWorstJobToHave,0.0,0.0,1.0,1.0,0.0,0.0


In [69]:
groupby_user_text_createdat = df_tweets_clear[duplicates_bool].groupby(['user_id', 'text','created_at'])

In [71]:
def get_first(list):
    return list

In [72]:
aggregated = groupby_user_text_createdat.agg({'retweet_count_conv':max,'reply_count_conv':max,'favorite_count_conv':max,'num_hashtags_conv':max,'num_urls_conv':max,'num_mentions_conv':max,'id':get_first})

['user_id', 'text', 'created_at']

In [74]:
aggregated.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1041126 entries, ('100043740', '#UofSC18 ', '2019-04-25 20:55:58') to ('999662546', 'http://t.co/HGjGUGG3w6', '2019-05-06 17:22:40')
Data columns (total 7 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   retweet_count_conv   1041126 non-null  float64
 1   reply_count_conv     1041126 non-null  float64
 2   favorite_count_conv  1041126 non-null  float64
 3   num_hashtags_conv    1041126 non-null  float64
 4   num_urls_conv        1041126 non-null  float64
 5   num_mentions_conv    1041126 non-null  float64
 6   id                   1041126 non-null  object 
dtypes: float64(6), object(1)
memory usage: 79.5+ MB


## Data la presenza di molteplici valori nulli tra i duplicati secondo le colonne [id, user_id, text] verifichiamo la corrispondenza di tweet con lo user_id non nullo con la tabella users

In [29]:
df_nonnull_users = df_tweets_dupl[df_tweets_dupl['user_id'].isnull() == False] # Solo 12 user_id sono avvalorati nei duplicati
df_nonnull_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 4515945 to 13484983
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              12 non-null     object
 1   user_id         12 non-null     object
 2   retweet_count   5 non-null      object
 3   reply_count     3 non-null      object
 4   favorite_count  2 non-null      object
 5   num_hashtags    0 non-null      object
 6   num_urls        2 non-null      object
 7   num_mentions    0 non-null      object
 8   created_at      12 non-null     object
 9   text            0 non-null      object
dtypes: object(10)
memory usage: 1.0+ KB


In [30]:
df_nonnull_users.user_id = pd.to_numeric(df_nonnull_users.user_id) # transform column to int values from object
df_nonnull_users.merge(df_users, left_on='user_id', right_on='id') # merge with users table

C:\Users\al\AppData\Local\Temp\ipykernel_15108\3993844464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonnull_users.user_id = pd.to_numeric(df_nonnull_users.user_id) # transform column to int values from object


,id_x,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at_x,text,id_y,name,lang,bot,created_at_y,statuses_count
0,vada,331806614,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-16 23:49:32,NaN,331806614,Marty Balin,en,1,2016-07-09 19:01:19,12375.0
1,ptb,467116841,0,0.0,0.0,NaN,NaN,NaN,2019-06-17 09:34:56,NaN,467116841,Gabriele Danno,it,1,2017-01-19 04:08:16,61.0
2,efh,1573156616,0,NaN,NaN,NaN,NaN,NaN,2020-03-07 04:35:15,NaN,1573156616,Tiny Tibo,fr,0,2018-07-08 16:27:04,27105.0
3,oby,114910555,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-15 03:51:40,NaN,114910555,Señor Grande,en,0,2015-02-19 00:51:08,10212.0
4,yfj,2445641666,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-29 23:52:04,NaN,2445641666,Barstool Iowa State,en,0,2019-04-17 15:59:24,1515.0
5,djq,70142270,NaN,NaN,NaN,NaN,0.0,NaN,2020-04-23 12:43:47,NaN,70142270,Morgan Joseph,en,0,2014-09-01 15:31:01,12171.0
6,gnn,24295753,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-15 15:12:05,NaN,24295753,Nick (Rosie),en,0,2014-03-16 00:53:21,18699.0
7,fkn,1978992043,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-21 23:10:07,NaN,1978992043,-,en-gb,0,2018-10-23 14:43:13,33905.0
8,vam,1499931151,1222,NaN,0.0,NaN,1.0,NaN,2020-03-15 14:54:04,NaN,1499931151,Bojan,en,0,2018-06-13 01:16:32,26559.0
9,tgg,576226470,0,0.0,NaN,NaN,NaN,NaN,2018-11-28 13:28:02,NaN,576226470,Dicerie,en,1,2017-05-12 11:13:08,2000.0


All users have a correspondance with users in the users file 

## Eliminazione delle righe ripetute secondo il pattern sopracitato aventi lo user_id nullo

In [31]:
df_tweets_more_clear = df_tweets_clear[df_tweets_clear.duplicated(subset=['id', 'user_id','text'])]

In [32]:
df_tweets_more_clear.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261 entries, 280761 to 13607558
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              261 non-null    object
 1   user_id         12 non-null     object
 2   retweet_count   142 non-null    object
 3   reply_count     51 non-null     object
 4   favorite_count  76 non-null     object
 5   num_hashtags    0 non-null      object
 6   num_urls        56 non-null     object
 7   num_mentions    0 non-null      object
 8   created_at      261 non-null    object
 9   text            0 non-null      object
dtypes: object(10)
memory usage: 22.4+ KB


Check null user_id values

In [33]:
df_tweets_more_clear[df_tweets_more_clear['user_id'].isnull() == True]

,id,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text
280761,mxi,NaN,0,NaN,0.0,NaN,1.0,NaN,2019-09-14 20:47:29,NaN
515406,mxi,NaN,2,NaN,0.0,NaN,0.0,NaN,2020-03-22 04:55:02,NaN
573128,srq,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2020-02-24 22:11:05,NaN
1706009,xiu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-21 14:12:26,NaN
1787554,crks,NaN,0,NaN,NaN,NaN,NaN,NaN,2020-04-17 16:52:26,NaN
...,...,...,...,...,...,...,...,...,...,...
13554922,cxz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-13 07:19:42,NaN
13557755,ngd,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2019-11-01 16:21:42,NaN
13561775,qyt,NaN,0,NaN,0.0,NaN,0.0,NaN,2019-11-03 11:50:00,NaN
13593848,uru,NaN,0,NaN,NaN,NaN,NaN,NaN,2020-03-27 01:57:35,NaN


Drop all records where user_id is null

In [34]:
df_tweets_single =df_tweets_clear.drop(df_tweets_more_clear[df_tweets_more_clear['user_id'].isnull() == True].index)
df_tweets_single.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11712348 entries, 0 to 13664695
Data columns (total 10 columns):
 #   Column          Non-Null Count     Dtype 
---  ------          --------------     ----- 
 0   id              11712346 non-null  object
 1   user_id         11495314 non-null  object
 2   retweet_count   11275326 non-null  object
 3   reply_count     11064671 non-null  object
 4   favorite_count  11064981 non-null  object
 5   num_hashtags    10655073 non-null  object
 6   num_urls        11063920 non-null  object
 7   num_mentions    10858432 non-null  object
 8   created_at      11712348 non-null  object
 9   text            11182415 non-null  object
dtypes: object(10)
memory usage: 982.9+ MB


Take all users in duplicates that have a user_id that is not null

In [35]:
df_nonnull_users = df_tweets_dupl[df_tweets_dupl['user_id'].isnull() == False]
df_nonnull_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 4515945 to 13484983
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              12 non-null     object
 1   user_id         12 non-null     object
 2   retweet_count   5 non-null      object
 3   reply_count     3 non-null      object
 4   favorite_count  2 non-null      object
 5   num_hashtags    0 non-null      object
 6   num_urls        2 non-null      object
 7   num_mentions    0 non-null      object
 8   created_at      12 non-null     object
 9   text            0 non-null      object
dtypes: object(10)
memory usage: 1.0+ KB


In [36]:
df_nonnull_users.user_id = pd.to_numeric(df_nonnull_users.user_id) # transform column to int values from object
df_nonnull_users.merge(df_users, left_on='user_id', right_on='id') # merge with users table

C:\Users\al\AppData\Local\Temp\ipykernel_15108\3993844464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonnull_users.user_id = pd.to_numeric(df_nonnull_users.user_id) # transform column to int values from object


,id_x,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at_x,text,id_y,name,lang,bot,created_at_y,statuses_count
0,vada,331806614,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-16 23:49:32,NaN,331806614,Marty Balin,en,1,2016-07-09 19:01:19,12375.0
1,ptb,467116841,0,0.0,0.0,NaN,NaN,NaN,2019-06-17 09:34:56,NaN,467116841,Gabriele Danno,it,1,2017-01-19 04:08:16,61.0
2,efh,1573156616,0,NaN,NaN,NaN,NaN,NaN,2020-03-07 04:35:15,NaN,1573156616,Tiny Tibo,fr,0,2018-07-08 16:27:04,27105.0
3,oby,114910555,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-15 03:51:40,NaN,114910555,Señor Grande,en,0,2015-02-19 00:51:08,10212.0
4,yfj,2445641666,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-29 23:52:04,NaN,2445641666,Barstool Iowa State,en,0,2019-04-17 15:59:24,1515.0
5,djq,70142270,NaN,NaN,NaN,NaN,0.0,NaN,2020-04-23 12:43:47,NaN,70142270,Morgan Joseph,en,0,2014-09-01 15:31:01,12171.0
6,gnn,24295753,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-15 15:12:05,NaN,24295753,Nick (Rosie),en,0,2014-03-16 00:53:21,18699.0
7,fkn,1978992043,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-21 23:10:07,NaN,1978992043,-,en-gb,0,2018-10-23 14:43:13,33905.0
8,vam,1499931151,1222,NaN,0.0,NaN,1.0,NaN,2020-03-15 14:54:04,NaN,1499931151,Bojan,en,0,2018-06-13 01:16:32,26559.0
9,tgg,576226470,0,0.0,NaN,NaN,NaN,NaN,2018-11-28 13:28:02,NaN,576226470,Dicerie,en,1,2017-05-12 11:13:08,2000.0


In [37]:
for attr in ['retweet_count','reply_count','favorite_count','num_hashtags','num_urls','num_mentions']:
    cazzo = df_tweets_single[df_tweets_single.duplicated(subset=['id', 'user_id', 'text', attr])]
    print(attr+'\n'+str(cazzo.shape[0])+'\n')
    print(cazzo)

retweet_count
7

           id     user_id retweet_count reply_count favorite_count  \
6978627   oby   114910555           NaN         NaN            NaN   
7509770   yfj  2445641666           NaN         NaN            NaN   
9354011   djq    70142270           NaN         NaN            NaN   
10231019  gnn    24295753           NaN         NaN            NaN   
11050094  fkn  1978992043           NaN         NaN            NaN   
12987829  tgg   576226470             0         0.0            NaN   
13484983  gaj   251905557           NaN         NaN            NaN   

         num_hashtags num_urls num_mentions           created_at text  
6978627           NaN      NaN          NaN  2020-03-15 03:51:40  NaN  
7509770           NaN      NaN          NaN  2019-04-29 23:52:04  NaN  
9354011           NaN      0.0          NaN  2020-04-23 12:43:47  NaN  
10231019          NaN      NaN          NaN  2020-03-15 15:12:05  NaN  
11050094          NaN      NaN          NaN  2020-03-21 23:10:

As this print shows, most of duplicates have missing values. If we exclude `retweet_counts`, the duplicates are the same.

In [38]:
df_tweets_single[df_tweets_single.duplicated(subset=['id', 'user_id', 'text', 'reply_count','favorite_count','num_hashtags','num_urls','num_mentions'], keep=False)].sort_values(by='id')

,id,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text
9354011,djq,70142270,NaN,NaN,NaN,NaN,0.0,NaN,2020-04-23 12:43:47,NaN
7858198,djq,70142270,NaN,NaN,NaN,NaN,0.0,NaN,2019-06-27 13:38:06,NaN
5123152,efh,1573156616,x0c7,NaN,NaN,NaN,NaN,NaN,2020-03-12 02:12:50,NaN
6445718,efh,1573156616,0,NaN,NaN,NaN,NaN,NaN,2020-03-07 04:35:15,NaN
8424121,fkn,1978992043,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-03 16:24:47,NaN
11050094,fkn,1978992043,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-21 23:10:07,NaN
13484983,gaj,251905557,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-25 17:51:35,NaN
13074603,gaj,251905557,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-04 23:35:19,NaN
10231019,gnn,24295753,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-15 15:12:05,NaN
8474665,gnn,24295753,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-14 00:09:55,NaN


In [39]:
df_tweets_single.to_csv(sep = '#', path_or_buf = DATA_PATH+'tweets_no_dupl.csv', index = False)

From now on we assume that the records in the dataset are not duplcated.